In [1]:
!pip install transformers[torch]
!pip install SentencePiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

model_checkpoint = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)
model = model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


cuda


In [3]:
# Demo inference

processed_input = """""
question: What does increased oxygen concentrations in the patient’s
lungs displace? context: Hyperbaric (high-pressure) medicine uses special
oxygen chambers to increase the partial pressure of O 2 around the patient
and, when needed, the medical staff. Carbon monoxide poisoning, gas gangrene,
and decompression sickness (the ’bends’) are sometimes treated using these
devices. Increased O 2 concentration in the lungs helps to displace carbon
monoxide from the heme group of hemoglobin. Oxygen gas is poisonous to the
anaerobic bacteria that cause gas gangrene, so increasing its partial pressure
helps kill them. Decompression sickness occurs in divers who decompress too
quickly after a dive, resulting in bubbles of inert gas, mostly nitrogen and
helium, forming in their blood. Increasing the pressure of O 2 as soon as
possible is part of the treatment.
"""

inputs = tokenizer(processed_input,
                                        max_length=200,
                                        padding="max_length",
                                        truncation=True,
                                        pad_to_max_length=True,
                                        add_special_tokens=True)

input_ids = torch.tensor(inputs.input_ids, dtype=torch.long).to(device).unsqueeze(0)
attention_mask = torch.tensor(inputs.attention_mask, dtype=torch.long).to(device).unsqueeze(0)

outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


carbon monoxide


In [4]:
# Demo inference on SQL query

processed_input = """""
question: Get all students information from table Students? context: CREATE TABLE students(id);
"""

inputs = tokenizer(processed_input,
                                        max_length=200,
                                        padding="max_length",
                                        truncation=True,
                                        pad_to_max_length=True,
                                        add_special_tokens=True)

input_ids = torch.tensor(inputs.input_ids, dtype=torch.long).to(device).unsqueeze(0)
attention_mask = torch.tensor(inputs.attention_mask, dtype=torch.long).to(device).unsqueeze(0)

outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

CREATE TABLE students(id);


In [5]:
# Import Dataset
from datasets import load_dataset

dataset = load_dataset("b-mc2/sql-create-context")
dataset2 = load_dataset("Clinton/Text-to-sql-v1")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 78577
    })
})

In [6]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text'],
        num_rows: 262208
    })
})

In [7]:
dataset["train"][0]

{'question': 'How many heads of the departments are older than 56 ?',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'context': 'CREATE TABLE head (age INTEGER)'}

In [8]:
# Convert to dataframe
import pandas as pd
df = pd.DataFrame(dataset["train"])
df.head()

,question,answer,context
0,How many heads of the departments are older th...,SELECT COUNT(*) FROM head WHERE age > 56,CREATE TABLE head (age INTEGER)
1,"List the name, born state and age of the heads...","SELECT name, born_state, age FROM head ORDER B...","CREATE TABLE head (name VARCHAR, born_state VA..."
2,"List the creation year, name and budget of eac...","SELECT creation, name, budget_in_billions FROM...","CREATE TABLE department (creation VARCHAR, nam..."
3,What are the maximum and minimum budget of the...,"SELECT MAX(budget_in_billions), MIN(budget_in_...",CREATE TABLE department (budget_in_billions IN...
4,What is the average number of employees of the...,SELECT AVG(num_employees) FROM department WHER...,CREATE TABLE department (num_employees INTEGER...


In [9]:
df2 = pd.DataFrame(dataset2["train"])
df2 = df2.drop(columns=['source', 'text'])
df2 = df2.rename(columns={"instruction": "question", "input": "context", "response": "answer"})
df2.head()

,question,context,answer
0,Name the home team for carlton away team,CREATE TABLE table_name_77 (\n home_team VA...,SELECT home_team FROM table_name_77 WHERE away...
1,what will the population of Asia be when Latin...,"CREATE TABLE table_22767 (\n ""Year"" real,\n...","SELECT ""Asia"" FROM table_22767 WHERE ""Latin Am..."
2,How many faculty members do we have for each g...,"CREATE TABLE Student (\n StuID INTEGER,\n ...","SELECT Sex, COUNT(*) FROM Faculty GROUP BY Sex..."
3,List the record of 0-1 from the table?,CREATE TABLE table_14656147_2 (\n week VARC...,SELECT week FROM table_14656147_2 WHERE record...
4,"Which silver has a Gold smaller than 12, a Ran...",CREATE TABLE table_name_24 (\n silver VARCH...,SELECT silver FROM table_name_24 WHERE gold < ...


In [10]:
# Merge 2 datasets
df = pd.concat([df, df2], axis=0, ignore_index=True)

In [11]:
df.shape

(340785, 3)

In [12]:
# Tokenize the dataset
# Example of tokenization the input


example_row = df.iloc[0]
print("query for: " + example_row["question"] )
print("tables: " + example_row["context"])
example_input = tokenizer(
    example_row["question"],
    example_row["context"],
    max_length=128,
    truncation=True,
    padding="max_length",
)
print("input_ids: " + str(example_input.input_ids))

# Number of tokens that are not padding token
sum([1 for id in example_input.input_ids if id != 0])

question: How many heads of the departments are older than 56 ?
context: CREATE TABLE head (age INTEGER)
input_ids: [571, 186, 7701, 13, 8, 10521, 33, 2749, 145, 11526, 3, 58, 1, 205, 4386, 6048, 332, 17098, 819, 41, 545, 3, 21342, 17966, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


26

In [13]:
# Example of tokenization the output
example_output = tokenizer(
    example_row["answer"],
    max_length=128,
    truncation=True,
    padding="max_length",
)
print("Answer: " + example_row["answer"])
print("output_ids: " + str(example_output.input_ids))

Answer: SELECT COUNT(*) FROM head WHERE age > 56
output_ids: [3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819, 549, 17444, 427, 1246, 2490, 11526, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
class QADataset:
  def __init__(self, data, tokenizer,  q_len, t_len):
    self.data = data
    self.q_len = q_len
    self.t_len = t_len
    self.tokenizer = tokenizer
    self.questions = data["question"]
    self.contexts = data["context"]
    self.answers = data["answer"]
  def __len__(self):
    return len(self.questions)
  def __getitem__(self, index):
    question = self.questions[index]
    context = self.contexts[index]
    answer = self.answers[index]

    # Tokenized input and target
    tokenized_question = self.tokenizer(f"query for: {question}",
                                        f"tables: {context}" ,
                                        max_length=self.q_len,
                                        padding="max_length",
                                        truncation=True,
                                        pad_to_max_length=True,
                                        add_special_tokens=True)
    tokenized_answer = self.tokenizer(answer,
                                      max_length=self.t_len,
                                      padding="max_length",
                                      truncation=True,
                                      pad_to_max_length=True,
                                      add_special_tokens=True)
    labels = torch.tensor(tokenized_answer["input_ids"], dtype=torch.long)
    labels[labels == 0] = -100

    return {
        "input_ids": torch.tensor(tokenized_question["input_ids"], dtype=torch.long),
        "attention_mask": torch.tensor(tokenized_question["attention_mask"], dtype=torch.long),
        "labels": labels,
        "decoder_attention_mask": torch.tensor(tokenized_answer["attention_mask"], dtype=torch.long)
    }

In [15]:
# Split the dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler


train_data, validation_data = train_test_split(df, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(validation_data.index)

print(train_data.index)
train_data.shape

Int64Index([253338,  91930, 287739,  93788,   1642, 272746, 195916, 210025,
             62710, 194927,
            ...
             87498, 137337,  54886, 207892, 110268, 119879, 259178, 131932,
            146867, 121958],
           dtype='int64', length=272628)


(272628, 3)

In [16]:
validation_data.shape

(68157, 3)

In [17]:
train_data.head()

,question,answer,context
253338,What is the inscription with John Vanbrugh lis...,"SELECT ""Inscription"" FROM table_8986 WHERE ""Id...","CREATE TABLE table_8986 (\n ""Side"" text,\n ..."
91930,Who was the home team when the away team score...,"SELECT ""Home team score"" FROM table_33414 WHER...","CREATE TABLE table_33414 (\n ""Home team"" te..."
287739,What was the Attendance when Lincoln City was ...,"SELECT COUNT(""Attendance"") FROM table_34045 WH...","CREATE TABLE table_34045 (\n ""Date"" text,\n..."
93788,how many days has passed since patient 31088 h...,"SELECT 1 * (STRFTIME('%j', CURRENT_TIME()) - S...",CREATE TABLE outputevents (\n row_id number...
1642,Which committees have delegates from both demo...,SELECT T1.Committee FROM election AS T1 JOIN p...,"CREATE TABLE election (Committee VARCHAR, Part..."


In [18]:
# Create dataloader

qa_dataset = QADataset(df, tokenizer, 200, 128)


BATCH_SIZE = 32

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [19]:
# Init the training params
from tqdm import tqdm
from torch.optim import Adam

train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0yc

learning_rate = 3e-5
epochs = 2
optimizer = Adam(model.parameters(), lr=learning_rate)

epoch_train_loss = []
epoch_val_loss = []

In [20]:
# Train the model

for epoch in range(epochs):
  # Train the model
  model.train()
  for batch in tqdm(train_loader, desc="Training batches"):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    decoder_attention_mask = batch["decoder_attention_mask"].to(device)

    batch_outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask)

    optimizer.zero_grad()
    batch_outputs.loss.backward()
    optimizer.step()

    train_loss += batch_outputs.loss.item()

    train_batch_count += 1

  # Evaluate the model
  model.eval()
  for batch in tqdm(val_loader, desc="Validation batches"):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    decoder_attention_mask = batch["decoder_attention_mask"].to(device)

    batch_outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask)
    optimizer.zero_grad()
    batch_outputs.loss.backward()
    optimizer.step()

    val_loss += batch_outputs.loss.item()
    val_batch_count += 1

  epoch_train_loss.append(train_loss / train_batch_count)
  epoch_val_loss.append(val_loss / val_batch_count)
  print(f"Epoch {epoch + 1}: Train loss: {train_loss / train_batch_count}, Validation loss: {val_loss / val_batch_count}")



Streaming output truncated to the last 5000 lines.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Training batches:  91%|█████████▏| 7782/8520 [1:36:58<09:06,  1.35it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some 

Epoch 1: Train loss: 0.7177304073486109, Validation loss: 0.12348765716516356


Streaming output truncated to the last 5000 lines.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Training batches:  91%|█████████ | 7772/8520 

Epoch 2: Train loss: 0.5358534773037782, Validation loss: 0.10472056783884413


In [21]:
print(epoch_train_loss)
print(epoch_val_loss)

[0.7177304073486109, 0.5358534773037782]
[0.12348765716516356, 0.10472056783884413]


In [22]:
model.save_pretrained("text2sql")
tokenizer.save_pretrained("text2sql_tokenizer")


('text2sql_tokenizer/tokenizer_config.json',
 'text2sql_tokenizer/special_tokens_map.json',
 'text2sql_tokenizer/spiece.model',
 'text2sql_tokenizer/added_tokens.json')

In [31]:
def predict(context, question):
  inputs = tokenizer(f"query for: {question}? ",
                                        f"tables: {context}" ,
                                        max_length=200,
                                        padding="max_length",
                                        truncation=True,
                                        pad_to_max_length=True,
                                        add_special_tokens=True)

  input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(device).unsqueeze(0)
  attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(device).unsqueeze(0)

  outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
  answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)
  return answer


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


'uses special oxygen chambers to increase the concentration of O 2 in the lungs and,'

In [40]:
context = """""
CREATE TABLE students (id INT ,name Text, class_id INT); CREATE TABLE classes (id INT, name TEXT);
"""
question = "count number of students in all classes"

answer = predict(context, question)
answer

'SELECT COUNT(*) FROM students AS T1 JOIN classes AS T'

In [39]:
question = "get id, name of all classes"

answer = predict(context, question)
answer

'SELECT id, name FROM classes ORDER BY name'

In [44]:
# Try loading the model from trained
from transformers import T5Config

# Load configuration
config = T5Config.from_json_file("/content/text2sql/config.json")

# Load model
model_test = T5ForConditionalGeneration.from_pretrained("text2sql").to(device)
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")


def predict_test(context, question):
  inputs = tokenizer(f"query for: {question}? ",
                                        f"tables: {context}" ,
                                        max_length=200,
                                        padding="max_length",
                                        truncation=True,
                                        pad_to_max_length=True,
                                        add_special_tokens=True)

  input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(device).unsqueeze(0)
  attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(device).unsqueeze(0)

  outputs = model_test.generate(input_ids=input_ids, attention_mask=attention_mask)
  answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)
  return answer

context = """""
CREATE TABLE students (id INT ,name Text, class_id INT); CREATE TABLE classes (id INT, name TEXT);
"""
question = "count number of students in all classes"

answer = predict_test(context, question)
answer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'SELECT COUNT(*) FROM students AS T1 JOIN classes AS T'